# Lab 3 — recommender systems

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from operator import add
from operator import itemgetter

%matplotlib inline
plt.style.use("ggplot")

What does the data look like?

In [ ]:
!hdfs dfs -cat /ix/ml-20m/ratings.txt | tail -n 2

#### Exercise 3.4

In [ ]:
ratings = sc.textFile("/ix/ml-20m/ratings.txt").map(json.loads)
uidOne = ratings.map(lambda x : (x['userId'], 1)) #creates a tuple (userId, 1)
movidOne = ratings.map(lambda x : (x['movieId'], 1)) #creates a tuple (userId, 1)

#add values of all tuples with same key. Gives tuple (Id, occurences of this userId)
sortUser = sorted(uidOne.reduceByKey(add).collect()) 
sortMovie = sorted(movidOne.reduceByKey(add).collect()) 

In [ ]:
#changes tuples (Id, occurence) to a list of occurences (index of vector = id of user/film)
tableUsers = np.zeros(len(sortUser))
tableMovies = np.zeros(len(sortMovie))
for i in range(len(sortUser)):
    tableUsers[i] = sortUser[i][1]
for i in range(len(sortMovie)):
    tableMovies[i] = sortMovie[i][1]
    
#Plot data
plt.plot(sorted(tableUsers))
plt.xlabel('User ID sorted by number of ratings')
plt.ylabel('Number of films rated')
plt.show();

plt.plot(sorted(tableMovies))
plt.xlabel('Movie ID sorted by number of ratings')
plt.ylabel('Number of ratings received')
plt.show();

We immediately see the distribution is not uniform : some users rated barely a few films (min is around 20), when some others rated more that 9'000 films. 

Same applies for the films. Most films have almost no rating : first 20'000 films have less than 200 ratings and first 4'000 have 1 ratings. On the other hand, a handful of them have tens of thousands.

#### Exercise 3.5

In [ ]:
with open("my-ratings.txt", "r") as F: #open file
    newEntries = ",".join(F.read().split("\n")[:-1]) #one string of all entries
data = sc.parallelize(json.loads("["+newEntries+"]")) #interpret string as JSON and make pysparkRDD
joined = ratings.union(data) #fusion the datasets

#separate datasets w.r.t timestamp (~ 20%/80%)
validation = joined.filter(lambda entry : entry["timestamp"]%10 <= 1) 
training = joined.filter(lambda entry : entry['timestamp']%10 > 1)

#Making sure sizes are correct
totalsize = joined.count()
validationsize = validation.count()
trainingsize = training.count()
print("Training set represents", 100*trainingsize/totalsize,"% and validation set represents", 100*validationsize/totalsize, "%")

#### Exercise 3.6

In [ ]:
#transform to a list of ratings, then sum and divide by total size
global_avg = training.map(lambda entry : entry['rating']).fold(0, add)/trainingsize
print("Global average is", global_avg)
# numratingsPerUser
# ubias = training.map(lambda entry : entry['rating']-global_avg).fold(0, add)/
# ibias = training.map(lambda entry : entry[rating-])